In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

In [ ]:
# Histogram verilerini tanımlama
histogram = {
    100: 12,
    101: 18,
    102: 32,
    103: 48,
    104: 52,
    105: 65,
    106: 55,
    107: 42,
    108: 32,
    109: 16,
    110: 10,
    140: 5,
    141: 18,
    142: 25,
    143: 32,
    144: 40,
    145: 65,
    146: 43,
    147: 32,
    148: 20,
    149: 10,
    150: 4
}

In [ ]:
# Histogram verilerini yoğunluk ve piksel sayıları listelerine ayırma
yoğunluk_değerleri = np.array(list(histogram.keys()))
piksel_sayıları = np.array(list(histogram.values()))

In [ ]:
# Toplam piksel sayısını hesaplama
toplam_piksel = np.sum(piksel_sayıları)

In [ ]:
# Yoğunluk değerlerinin piksel sayılarını normalize etme
normalleştirilmiş_histogram = piksel_sayıları / toplam_piksel

In [ ]:
# Otsu eşikleme yöntemi
def otsu_eşikleme(yoğunluk_değerleri, normalleştirilmiş_histogram):
    maksimum_varyans = 0
    en_uygun_eşik_değeri = 0
    veri_listesi = []

    for eşik in yoğunluk_değerleri:
        # Sınıf 1 için ağırlık, ortalama ve varyans hesaplama
        ağırlık1 = np.sum(normalleştirilmiş_histogram[yoğunluk_değerleri <= eşik])
        ortalama1 = np.sum(yoğunluk_değerleri[yoğunluk_değerleri <= eşik] * normalleştirilmiş_histogram[yoğunluk_değerleri <= eşik]) / ağırlık1 if ağırlık1 > 0 else 0
        varyans1 = np.sum(((yoğunluk_değerleri[yoğunluk_değerleri <= eşik] - ortalama1) ** 2) * normalleştirilmiş_histogram[yoğunluk_değerleri <= eşik]) / ağırlık1 if ağırlık1 > 0 else 0
        
        # Sınıf 2 için ağırlık, ortalama ve varyans hesaplama
        ağırlık2 = np.sum(normalleştirilmiş_histogram[yoğunluk_değerleri > eşik])
        ortalama2 = np.sum(yoğunluk_değerleri[yoğunluk_değerleri > eşik] * normalleştirilmiş_histogram[yoğunluk_değerleri > eşik]) / ağırlık2 if ağırlık2 > 0 else 0
        varyans2 = np.sum(((yoğunluk_değerleri[yoğunluk_değerleri > eşik] - ortalama2) ** 2) * normalleştirilmiş_histogram[yoğunluk_değerleri > eşik]) / ağırlık2 if ağırlık2 > 0 else 0
        
        # Sınıflar arası varyansı hesaplama
        varyans_arası = ağırlık1 * ağırlık2 * (ortalama1 - ortalama2) ** 2

        # En yüksek varyansı bulma 
        if varyans_arası > maksimum_varyans:
            maksimum_varyans = varyans_arası
            en_uygun_eşik_değeri = eşik

        # Veri listesine ekleme
        veri_listesi.append((eşik, ağırlık1, ortalama1, varyans1, ağırlık2, ortalama2, varyans2, varyans_arası))

    return en_uygun_eşik_değeri, veri_listesi

In [ ]:
# Otsu yöntemi ile en uygun eşik değerini hesaplama
en_uygun_eşik_değeri, veri_listesi = otsu_eşikleme(yoğunluk_değerleri, normalleştirilmiş_histogram)
print(f"En uygun eşik değeri: {en_uygun_eşik_değeri}")

In [ ]:
# Tabloda gösterim için gerekli verileri toplama
eşik_değerleri = [v[0] for v in veri_listesi]
ağırlık1_değerleri = [v[1] for v in veri_listesi]
ortalama1_değerleri = [v[2] for v in veri_listesi]
varyans1_değerleri = [v[3] for v in veri_listesi]
ağırlık2_değerleri = [v[4] for v in veri_listesi]
ortalama2_değerleri = [v[5] for v in veri_listesi]
varyans2_değerleri = [v[6] for v in veri_listesi]
varyans_arası_değerleri = [v[7] for v in veri_listesi]

In [ ]:
# Histogramı ve eşik değerini görselleştirme
fig = plt.figure(figsize=(18, 12))
gs = GridSpec(3, 1, height_ratios=[3, 1, 2])

In [ ]:
ax1 = plt.subplot(gs[0])
ax1.bar(yoğunluk_değerleri, piksel_sayıları, width=1, edgecolor='black', align='center')
ax1.axvline(x=en_uygun_eşik_değeri, color='red', linestyle='--', label=f'Eşik Değeri: {en_uygun_eşik_değeri}')
ax1.set_title('Histogram ve Otsu Eşik Değeri')
ax1.set_xlabel('Yoğunluk Değerleri')
ax1.set_ylabel('Piksel Sayıları')
ax1.legend()

In [ ]:
# Eşik değerlerine göre hesaplamaların gösterimi
ax2 = plt.subplot(gs[1])
ax2.plot(eşik_değerleri, varyans_arası_değerleri, marker='o')
ax2.axvline(x=en_uygun_eşik_değeri, color='red', linestyle='--')
ax2.set_title('Eşik Değerlerine Göre Sınıflar Arası Varyans')
ax2.set_xlabel('Eşik Değeri')
ax2.set_ylabel('Sınıflar Arası Varyans')

In [ ]:
# Tablo için verileri hazırlama
cell_text = []
rows = ['Weight Background', 'Mean Background', 'Variance Background', 'Weight Foreground', 'Mean Foreground', 'Variance Foreground', 'Between Class Variance']
cols = [f'T={t}' for t in eşik_değerleri]

In [ ]:
for i in range(len(eşik_değerleri)):
    cell_text.append([
        f"{ağırlık1_değerleri[i]:.3f}",
        f"{ortalama1_değerleri[i]:.3f}",
        f"{varyans1_değerleri[i]:.3f}",
        f"{ağırlık2_değerleri[i]:.3f}",
        f"{ortalama2_değerleri[i]:.3f}",
        f"{varyans2_değerleri[i]:.3f}",
        f"{varyans_arası_değerleri[i]:.3f}"
    ])

In [ ]:
# Tabloyu oluşturma
ax3 = plt.subplot(gs[2])
table = ax3.table(cellText=cell_text, rowLabels=cols, colLabels=rows, cellLoc='center', loc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.2)
ax3.axis('off')

In [ ]:
plt.tight_layout()
plt.show()